# Table Metadata

Exploring the NYC Building Elevation and Subgrade (BES) Data Set in Python  

Author: Mark Bauer

In [1]:
import duckdb
import requests

In [2]:
# Printing verions of Python modules and packages with **watermark** - the IPython magic extension.
%reload_ext watermark
%watermark -v -p pandas,duckdb

Python implementation: CPython
Python version       : 3.8.13
IPython version      : 8.4.0

pandas: 1.4.3
duckdb: 0.10.0



In [3]:
url = 'https://data.cityofnewyork.us/api/views/bsin-59hv/rows.csv?accessType=DOWNLOAD'
file_name = 'data/bes-data.csv'

response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content of the response to a local file
    with open(file_name, 'wb') as f:
        f.write(response.content)
    print(f"CSV file {file_name} downloaded successfully.")
else:
    print("Failed to download CSV file.")

CSV file data/bes-data.csv downloaded successfully.


In [4]:
ls data/

bes-data.csv         policies-36047.json  policies-36085.json
bes-data.parquet     policies-36061.json
policies-36005.json  policies-36081.json


In [5]:
# create a DuckDB database instance
con = duckdb.connect()

# create table of the bes data
con.sql(
    """
    CREATE TABLE bes_data AS
    FROM read_csv('data/bes-data.csv')
    """
)

# examine count of rows
con.sql("SELECT COUNT(*) AS count_rows FROM bes_data").show()

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│     861876 │
└────────────┘



In [6]:
con.sql("DESCRIBE bes_data").to_df()

,column_name,column_type,null,key,default,extra
0,the_geom,VARCHAR,YES,None,None,None
1,bin,BIGINT,YES,None,None,None
2,bbl,BIGINT,YES,None,None,None
3,borough,BIGINT,YES,None,None,None
4,block,BIGINT,YES,None,None,None
5,lot,BIGINT,YES,None,None,None
6,address,VARCHAR,YES,None,None,None
7,z_grade,DOUBLE,YES,None,None,None
8,z_floor,DOUBLE,YES,None,None,None
9,subgrade,VARCHAR,YES,None,None,None


In [7]:
con.sql("SELECT count(*) AS count_columns FROM (DESCRIBE bes_data)").to_df()

,count_columns
0,26


In [8]:
sample_df = con.sql("SELECT * FROM bes_data LIMIT 5").to_df()

sample_df

,the_geom,bin,bbl,borough,block,lot,address,z_grade,z_floor,subgrade,...,longitude,pluto_bbl,Council,BoroCD,CTLabel,BoroCT2020,NTA2020,NTAName,CDTA2020,CDTAName
0,POINT (-74.22274561187417 40.52134422844183),5128004,5075340353,5,7534,353,78 SAVO LOOP,29.428,32.332,N,...,-74.222746,5075340353,51,503,226.01,5022601,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,SI03,SI03 South Shore (CD 3 Approximation)
1,POINT (-74.24179250549321 40.52875316810818),5155392,5075960125,5,7596,125,72 CHART LOOP,25.366,26.703,N,...,-74.241793,5075960125,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)
2,POINT (-74.24109012652094 40.528883822921635),5148808,5075960131,5,7596,131,40 TIDES LANE,36.172,38.902,N,...,-74.241090,5075960131,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)
3,POINT (-74.24649640323727 40.507045708438184),5088274,5079150042,5,7915,42,328 MAIN STREET,69.897,72.459,N,...,-74.246496,5079150042,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)
4,POINT (-74.24198032271697 40.51005666426356),5087850,5078680123,5,7868,123,309 SLEIGHT AVENUE,74.907,79.122,N,...,-74.241980,5078680123,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)


In [9]:
sample_df.iloc[:, :15]

,the_geom,bin,bbl,borough,block,lot,address,z_grade,z_floor,subgrade,notes1,notes2,notes3,x,y
0,POINT (-74.22274561187417 40.52134422844183),5128004,5075340353,5,7534,353,78 SAVO LOOP,29.428,32.332,N,Property was Successfully Measured,None,None,922321.468334,129295.106289
1,POINT (-74.24179250549321 40.52875316810818),5155392,5075960125,5,7596,125,72 CHART LOOP,25.366,26.703,N,Property was Successfully Measured,Attached Garage to Living Space,None,917033.446429,132008.386296
2,POINT (-74.24109012652094 40.528883822921635),5148808,5075960131,5,7596,131,40 TIDES LANE,36.172,38.902,N,Property was Successfully Measured,None,None,917228.833178,132055.448538
3,POINT (-74.24649640323727 40.507045708438184),5088274,5079150042,5,7915,42,328 MAIN STREET,69.897,72.459,N,Property was Successfully Measured,None,None,915703.546202,124103.533995
4,POINT (-74.24198032271697 40.51005666426356),5087850,5078680123,5,7868,123,309 SLEIGHT AVENUE,74.907,79.122,N,Property was Successfully Measured,None,None,916962.418356,125196.988173


In [10]:
sample_df.iloc[:, 15:]

,latitude,longitude,pluto_bbl,Council,BoroCD,CTLabel,BoroCT2020,NTA2020,NTAName,CDTA2020,CDTAName
0,40.521344,-74.222746,5075340353,51,503,226.01,5022601,SI0304,Annadale-Huguenot-Prince's Bay-Woodrow,SI03,SI03 South Shore (CD 3 Approximation)
1,40.528753,-74.241793,5075960125,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)
2,40.528884,-74.241090,5075960131,51,503,226.02,5022602,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)
3,40.507046,-74.246496,5079150042,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)
4,40.510057,-74.241980,5078680123,51,503,244.01,5024401,SI0305,Tottenville-Charleston,SI03,SI03 South Shore (CD 3 Approximation)


In [11]:
con.sql("SUMMARIZE bes_data").to_df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,the_geom,VARCHAR,POINT (-73.70028493810709 40.73982670098595),POINT (-74.25526629488128 40.507487398894796),859938,None,None,None,None,None,861876,0.00
1,bin,BIGINT,1000003,5799523,859098,3610723.244786953,1053306.792880408,3089944,4024983,4258005,861876,0.00
2,bbl,BIGINT,1090961,5999999999,796558,3517132892.0941052,1034376481.7873905,3031367116,4010857983,4105529403,861876,0.00
3,borough,BIGINT,1,5,5,3.464081218274112,1.0262401210524328,3,4,4,861876,0.00
4,block,BIGINT,1,99999,13737,5310.378501522842,3725.9737014125576,2295,4877,7491,861876,0.00
5,lot,BIGINT,0,9999,2228,152.17001401593734,835.8620785492647,18,38,64,861876,0.00
6,address,VARCHAR,1 1 PLACE,YORK AVENUE,799297,None,None,None,None,None,861876,1.97
7,z_grade,DOUBLE,-13.096,402.839,154473,53.91501240781705,42.19289508297017,23.285157509641568,44.35556779154936,72.59686703547162,861876,0.00
8,z_floor,DOUBLE,0.0,404.397,157573,58.1137039481319,42.56067271472611,27.443648519240398,48.40061408306,77.00702070569508,861876,0.00
9,subgrade,VARCHAR,N,Y,2,None,None,None,None,None,861876,0.00


In [12]:
summarize_df = con.sql("SUMMARIZE bes_data").to_df()

summarize_df = summarize_df.loc[:, ['column_name', 'null_percentage']].sort_values(by='null_percentage', ascending=False)

summarize_df

,column_name,null_percentage
11,notes2,76.51
12,notes3,45.24
6,address,1.97
0,the_geom,0.00
15,latitude,0.00
24,CDTA2020,0.00
23,NTAName,0.00
22,NTA2020,0.00
21,BoroCT2020,0.00
20,CTLabel,0.00


In [13]:
con.sql("COPY bes_data TO 'data/bes-data.parquet' (FORMAT PARQUET)")

In [14]:
ls data/

bes-data.csv         policies-36047.json  policies-36085.json
bes-data.parquet     policies-36061.json
policies-36005.json  policies-36081.json


In [15]:
!du -h data/*

301M	data/bes-data.csv
 96M	data/bes-data.parquet
 97M	data/policies-36005.json
416M	data/policies-36047.json
112M	data/policies-36061.json
592M	data/policies-36081.json
289M	data/policies-36085.json


In [16]:
!rm data/bes-data.csv

In [17]:
ls data/

bes-data.parquet     policies-36047.json  policies-36081.json
policies-36005.json  policies-36061.json  policies-36085.json


In [18]:
# create a DuckDB database instance
con = duckdb.connect()

# examine count of rows
con.sql("SELECT COUNT(*) AS count_rows FROM read_parquet('data/bes-data.parquet')").show()

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│     861876 │
└────────────┘



In [19]:
con.close()